### VCF loading

In [2]:
#load packages
library(ggplot2)
library(plyr)
library(tidyverse)
library(grid)
library(gridExtra)
library(ggrepel)
library(RColorBrewer)
library(glue)
library(cowplot)
library(UpSetR)
library(pheatmap)
library(vcfR)
library(igraph)
library(ggpubr)
library(readr)
library(VennDiagram)

Warning message:
“package ‘ggplot2’ was built under R version 4.0.2”
Warning message:
“package ‘tidyverse’ was built under R version 4.0.2”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1
✔ purrr   0.3.4     

Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘readr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
Warning message:
“package ‘forcats’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()  

In [8]:
ntlist <- c('A','T','C','G','-')

In [7]:
project_folder <- '/Users/roderae/'
mva_folder <- glue("{project_folder}/compare_callers/2022")

In [9]:
setwd(mva_folder)
# grab files from each caller folder
vcfs.t <- Sys.glob("**/timo/*.vcf")
vcfs.fs <- Sys.glob("**/filtered_snps/*.vcf")
vcfs.i <- Sys.glob("**/ivar/*.vcf")
vcfs.fb <- Sys.glob("**/freebayes/*.vcf") 
vcfs.v <- Sys.glob("**/varscan/*.vcf")
vcfs.m <- Sys.glob("**/mutect2/*default-unfiltered.vcf")
vcfs.l <- Sys.glob("**/lofreq/*.vcf")

# make a single list for callers that can be loaded together
vcf_lists <- as.list(c(vcfs.fs,vcfs.i))

#### Lofreq VCF files  

lofreq uses an outdated VCF format and thus cannot be loaded using VCF R

In [51]:
# make a data frame to hold lofreq vcf info 
vcf.lofreq <- data.frame()

# for each file, read in the table
# add a header row
# separate the info by semi-colon so it can be extracted
# get rid of definitions so data can be used for calculations
# make separate columns for tool and file name
for (filename in vcfs.l) {
    # read in a file
    vcf <- read.table(filename,stringsAsFactors = F)
    names(vcf) <- c("chrom","ntpos","ID","refnt","varnt","qual","filter","info")
    vcf <- vcf %>% separate(info, sep = ';', into = c("DP","AF","SB","DP4"))
    vcf$DP <- gsub("DP=","",vcf$DP)
    vcf$AF <- gsub("AF=","",vcf$AF)
    vcf$SB <- gsub("SB=","",vcf$SB)
    vcf$DP4 <- gsub("DP4=","",vcf$DP4)
    vcf$source_ID <- filename
    
    vcf <- separate(vcf, source_ID, sep = "/", 
               into = c("rep","tool","file"))

    vcf <- separate(vcf, file, sep = "_fixed", 
               into = c("source_ID","rest")) %>% 
           select(-c(rest)) %>% 
           droplevels()
    
    # round allele frequency to 4 digits
    vcf$DP = as.numeric(vcf$DP)
    vcf$AF = as.numeric(vcf$AF)
    vcf$AF <- round(vcf$AF, digits = 4)

    # select desired columns and name appropriately
    vcf <- vcf %>% select("tool","source_ID","ntpos","refnt","varnt","AF","DP","rep")
    names(vcf) <- c("tool","source_ID","ntpos","refnt","varnt","varfreq","totalcount","rep")
    
    # add to the lofreq data frame
    vcf.lofreq <- rbind(vcf.lofreq, vcf)
    
}

#### Varscan VCF files

In [52]:
# make data frame to hold varscan vcf files
vcf.varscan <- data.frame()

# for each varscan vcf file
for (filename in vcfs.v) {
    print(filename)
    # read in a file
    # convert to usable data frame
    # set source_ID column
    vcf <- read.vcfR(filename, verbose = FALSE)
    vcf_df <- vcfR2tidy(vcf,single_frame = TRUE)
    vcf_df <- as.data.frame(vcf_df[[1]])
    vcf_df$source_ID <- filename

    # add to varscan data frame
    vcf.varscan <- rbind(vcf.varscan, vcf_df)
    
}

# select desired columns
vcf.varscan <- vcf.varscan %>% select(source_ID, POS, REF, ALT, QUAL, FILTER, gt_AD, ADP, gt_AD, gt_DP)

# convert column class to numeric where applicable
vcf.varscan$gt_DP = as.numeric(vcf.varscan$gt_DP)
vcf.varscan$ADP = as.numeric(vcf.varscan$ADP)
# calculate frequency using AD and ADP
vcf.varscan <- vcf.varscan %>%
    mutate(FREQ = gt_AD/ADP)
vcf.varscan$FREQ <- round(vcf.varscan$FREQ, digits = 4)

# separate tool and file columns
vcf.varscan <- separate(vcf.varscan, source_ID, sep = "/", 
               into = c("rep","tool","file"))

vcf.varscan <- separate(vcf.varscan, file, sep = "_fixed", 
               into = c("source_ID","rest")) %>% 
           select(-c(rest)) %>% 
           droplevels()

# select and rename desired columns to match other tools
vcf.varscan <- vcf.varscan %>% select(tool, source_ID, POS, REF, ALT, QUAL, FILTER, FREQ, gt_AD, ADP, rep)
names(vcf.varscan) <- c("tool","source_ID","ntpos","refnt","varnt","qual","filter","varfreq","allele_depth","totalcount","rep")
vcf.varscan <- vcf.varscan %>% select(tool, source_ID, ntpos, refnt, varnt, varfreq, totalcount, rep)

[1] "rep1/varscan/694_fixed.cov19.sorted.merged_varscan.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element SDP

Extracting gt element DP

Extracting gt element RD

Extracting gt element AD

Extracting gt element FREQ

Extracting gt element PVAL

Extracting gt element RBQ

Extracting gt element ABQ

Extracting gt element RDF

Extracting gt element RDR

Extracting gt element ADF

Extracting gt element ADR



[1] "rep1/varscan/OMS-CA-013_fixed.cov19.sorted.merged_varscan.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element SDP

Extracting gt element DP

Extracting gt element RD

Extracting gt element AD

Extracting gt element FREQ

Extracting gt element PVAL

Extracting gt element RBQ

Extracting gt element ABQ

Extracting gt element RDF

Extracting gt element RDR

Extracting gt element ADF

Extracting gt element ADR



[1] "rep1/varscan/OMS-CA-016_fixed.cov19.sorted.merged_varscan.vcf"


...

Extracting gt element GT

Extracting gt element GQ

Extracting gt element SDP

Extracting gt element DP

Extracting gt element RD

Extracting gt element AD

Extracting gt element FREQ

Extracting gt element PVAL

Extracting gt element RBQ

Extracting gt element ABQ

Extracting gt element RDF

Extracting gt element RDR

Extracting gt element ADF

Extracting gt element ADR



[1] "rep2/varscan/OMS-CA-629_fixed_varscan.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element SDP

Extracting gt element DP

Extracting gt element RD

Extracting gt element AD

Extracting gt element FREQ

Extracting gt element PVAL

Extracting gt element RBQ

Extracting gt element ABQ

Extracting gt element RDF

Extracting gt element RDR

Extracting gt element ADF

Extracting gt element ADR



[1] "rep2/varscan/OMS-CA-636_fixed_varscan.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element SDP

Extracting gt element DP

Extracting gt element RD

Extracting gt element AD

Extracting gt element FREQ

Extracting gt element PVAL

Extracting gt element RBQ

Extracting gt element ABQ

Extracting gt element RDF

Extracting gt element RDR

Extracting gt element ADF

Extracting gt element ADR



#### Mutect2 VCF files

In [53]:
# create data frame to hold mutect vcf files
vcf.mutect <- data.frame()

# for each mutect2 vcf file
for (filename in vcfs.m) {
    print(filename)
    # read in a file
    vcf <- read.vcfR(filename, verbose = FALSE)
    
    # convert to a standard dataframe
    vcf_df <- vcfR2tidy(vcf,single_frame = TRUE)
    # extract the data frame from the list
    vcf_df <- as.data.frame(vcf_df[[1]])

    # separate multiallelic positions from single allele positions
    # multi allelic positions will not be just A, C, T, or G
    multi <- vcf_df %>% filter(!ALT %in% ntlist)
    single <- vcf_df %>% filter(ALT %in% ntlist)

    # detect just rows where the alt allele positions contains a comma 
    # indels were not considered for this study
    # select necessary columns and filter out indels
    multi <- multi %>% filter(str_detect(multi$ALT, ",") == TRUE) %>% 
                       select(c(1:10),"gt_AD","gt_DP") %>% 
                       filter(REF %in% ntlist)

    # separate the allele depth column into the ref value, the depth of the first allele and the depth of the second allele
    multi <- separate(multi, gt_AD, sep = ',', into = c("refAF", "adp1", "adp2"))
    # convert all to numeric values
    multi$refAF <- as.numeric(multi$refAF)
    multi$adp1 <- as.numeric(multi$adp1)
    multi$adp2 <- as.numeric(multi$adp2)
    # calculate allele frequency by dividing the depth of each allele by the total depth at a given position
    multi <- multi %>% mutate(AF1 = adp1/gt_DP)
    multi <- multi %>% mutate(AF2 = adp2/gt_DP)
    # convert the allele frequencies to numeric values
    multi$AF1 <- as.numeric(multi$AF1)
    multi$AF2 <- as.numeric(multi$AF2)

    # separate the allele into nucleotide for the first and nucleotide for the second
    multi <- separate(multi, ALT, sep = ",", 
                       into = c("ALT1","ALT2"))
    
    # add a new row for each second allele where the first allele frequency goes with the first allele and second freq with the second allele
    # select desired columns
    multi <- multi %>% pivot_longer(cols = starts_with("ALT"),
                                    names_to = "var",
                                    values_to = "ALT") %>% 
                       mutate(AF=ifelse(var == "ALT1", AF1, AF2)) %>%
                       select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, AF, gt_DP)

    # select desired columns and get rid of indels
    single <- single %>% select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, gt_AD, gt_DP) %>% filter(REF %in% ntlist)

    # use allele depth column to calculate variant frequency for single allele positions
    single <- separate(single, gt_AD, sep = ',', into = c("major_dp","minor_dp"))
    single$minor_dp <- as.numeric(single$minor_dp)
    # variant freq is equal to number of reads with the alt nucleotide over the total number of reads
    single <- single %>%
                mutate(AF = minor_dp/gt_DP)
    single$AF <- as.numeric(single$AF)
    # round decimals to 4 digits
    single$AF <- round(single$AF, digits = 4)
    single <- single %>% select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, AF, gt_DP)
    
    # combine the multi allele positions back with the single allele positons
    vcf_df <- rbind(single, multi)
    vcf_df$source_ID <- filename
    
    # add to mutect data frame
    vcf.mutect <- rbind(vcf.mutect, vcf_df)
    
}

# separate tool and file columns
vcf.mutect <- separate(vcf.mutect, source_ID, sep = "/", 
               into = c("rep","tool","file"))

# get source_ID
vcf.mutect <- separate(vcf.mutect, file, sep = "_fixed", 
               into = c("source_ID","rest")) %>% 
           select(-c(rest)) %>% 
           droplevels()

# change class of columns where necessary
vcf.mutect$gt_DP = as.numeric(vcf.mutect$gt_DP)
vcf.mutect$AF = as.numeric(vcf.mutect$AF)
vcf.mutect$AF <- round(vcf.mutect$AF, digits = 4)

# select necessary columns
vcf.mutect <- vcf.mutect %>% select(tool, source_ID, POS, REF, ALT, AF, gt_DP, rep)

# set names of columns to match other tools
names(vcf.mutect) <- c("tool","source_ID","ntpos","refnt","varnt","varfreq","totalcount","rep")

[1] "rep1/mutect2/694_fixed.cov19.sorted.merged_mutect2-default-unfiltered.vcf"


Extracting gt element AD

Extracting gt element AF

Extracting gt element DP

Extracting gt element F1R2

Extracting gt element F2R1

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PGT

Extracting gt element PID

Extracting gt element PL

Extracting gt element PS

Extracting gt element SB



[1] "rep1/mutect2/OMS-CA-013_fixed.cov19.sorted.merged_mutect2-default-unfiltered.vcf"


Extracting gt element AD

Extracting gt element AF

Extracting gt element DP

Extracting gt element F1R2

Extracting gt element F2R1

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PGT

Extracting gt element PID

Extracting gt element PL

Extracting gt element PS

Extracting gt element SB



[1] "rep1/mutect2/OMS-CA-016_fixed.cov19.sorted.merged_mutect2-default-unfiltered.vcf"


...

Extracting gt element AD

Extracting gt element AF

Extracting gt element DP

Extracting gt element F1R2

Extracting gt element F2R1

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PGT

Extracting gt element PID

Extracting gt element PL

Extracting gt element PS

Extracting gt element SB

Warning message:
“Expected 3 pieces. Additional pieces discarded in 1 rows [12].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 1 rows [12].”


[1] "rep2/mutect2/OMS-CA-629_fixed_mutect2-default-unfiltered.vcf"


Extracting gt element AD

Extracting gt element AF

Extracting gt element DP

Extracting gt element F1R2

Extracting gt element F2R1

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PGT

Extracting gt element PID

Extracting gt element PL

Extracting gt element PS

Extracting gt element SB



[1] "rep2/mutect2/OMS-CA-636_fixed_mutect2-default-unfiltered.vcf"


Extracting gt element AD

Extracting gt element AF

Extracting gt element DP

Extracting gt element F1R2

Extracting gt element F2R1

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PGT

Extracting gt element PID

Extracting gt element PL

Extracting gt element PS

Extracting gt element SB



### Timo VCF files

In [54]:
# set up dataframe to hold vcf files for timo, haplotype caller and ivar
vcf.all.t <- data.frame()

for (filename in vcfs.t) {
    # read in a file
    vcf <- read.vcfR(filename, verbose = FALSE)
    # convert to a usable data frame
    vcf_df <- vcfR2tidy(vcf,single_frame = TRUE)
    vcf_df <- as.data.frame(vcf_df[[1]])
    
    # separate multiallelic positions from single allele positions
    multi <- vcf_df %>% filter(!ALT %in% ntlist)
    single <- vcf_df %>% filter(ALT %in% ntlist)
    
    multi <- distinct(multi)
    
    # separate alleles
    multi <- separate(multi, ALT, sep = ",", 
               into = c("ALT1","ALT2"))
    # separate allele frequencies
    multi <- separate(multi, AF, sep = ",",
                 into = c("AF1", "AF2"))
    
    # add a new row for each second allele
    multi <- multi %>% pivot_longer(cols = starts_with("ALT"),
                                    names_to = "var",
                                    values_to = "ALT") %>% 
                       mutate(AF=ifelse(var == "ALT1", AF1, AF2)) %>%
                       select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, AF, Indiv, gt_GT, gt_AD, gt_DP, gt_GQ, gt_GT_alleles)
    
    vcf_df <- rbind(single, multi)
    # make source ID column
    vcf_df$source_ID <- filename
    
    # select desired columns in order
    vcf_df <- vcf_df %>% select(source_ID, POS, REF, ALT, QUAL, FILTER, AF, gt_DP)
    
    # add to larger data frame
    vcf.all.t <- rbind(vcf.all.t, vcf_df)
    
}

# separate tool and file columns
vcf.all.t <- separate(vcf.all.t, source_ID, sep = "/", 
               into = c("rep","tool","file"))

# get source ID
vcf.all.t <- separate(vcf.all.t, file, sep = "_fixed", 
               into = c("source_ID","rest")) %>% 
           select(-c(rest)) %>% 
           droplevels()

# change class of columns where necessary
vcf.all.t$gt_DP = as.numeric(vcf.all.t$gt_DP)
vcf.all.t$AF = as.numeric(vcf.all.t$AF)
vcf.all.t$AF <- round(vcf.all.t$AF, digits = 4)

# set names to match other data frames
names(vcf.all.t) <- c("rep","tool","source_ID","ntpos","refnt","varnt","qual","filter","varfreq","totalcount")
# select columns
vcf.all.t <- vcf.all.t %>% select(tool, source_ID, ntpos, refnt, varnt, varfreq, totalcount, rep) %>% filter(!varnt == '-')

Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element AD

Extracting g

### Freebayes VCF files

In [55]:
# set up dataframe to hold vcf files for timo, haplotype caller and ivar
vcf.all.fb <- data.frame()

for (filename in vcfs.fb) {
    
    print(filename)
    # read in a file
    vcf <- read.vcfR(filename, verbose = FALSE)

    vcf_df <- vcfR2tidy(vcf,single_frame = TRUE)
    vcf_df <- as.data.frame(vcf_df[[1]])
    
    multi <- vcf_df %>% filter(!ALT %in% ntlist)
    single <- vcf_df %>% filter(ALT %in% ntlist)

    multi <- multi %>% filter(str_detect(multi$ALT, ",") == TRUE) %>% select(c(1:10),"RO","AO","gt_AD","gt_DP")

    multi <- separate(multi, gt_AD, sep = ',', into = c("refAF", "adp1", "adp2"))
    multi$refAF <- as.numeric(multi$refAF)
    multi$adp1 <- as.numeric(multi$adp1)
    multi$adp2 <- as.numeric(multi$adp2)
    multi <- multi %>% mutate(AF1 = adp1/gt_DP)
    multi <- multi %>% mutate(AF2 = adp2/gt_DP)
    multi$AF1 <- as.numeric(multi$AF1)
    multi$AF2 <- as.numeric(multi$AF2)

    multi <- separate(multi, ALT, sep = ",", 
               into = c("ALT1","ALT2"))

    multi <- multi %>% pivot_longer(cols = starts_with("ALT"),
                                    names_to = "var",
                                    values_to = "ALT") %>% 
                       mutate(AF=ifelse(var == "ALT1", AF1, AF2)) %>%
                       select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, AF, gt_DP)

    single <- single %>% select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, gt_AD, gt_DP)

    # use allele depth column to calculate variant frequency
    single <- separate(single, gt_AD, sep = ',', into = c("major_dp","minor_dp"))
    single$minor_dp <- as.numeric(single$minor_dp)
    # variant freq is equal to number of reads with the alt nucleotide over the total number of reads
    single <- single %>%
        mutate(AF = minor_dp/gt_DP)
    single$AF <- as.numeric(single$AF)
    # round decimals to 4 digits
    single$AF <- round(single$AF, digits = 4)
    single <- single %>% select(CHROM, POS, ID, REF, ALT, QUAL, FILTER, AF, gt_DP)
    
    vcf_df <- rbind(single, multi)
    
     # make source ID column
    vcf_df$source_ID <- filename
    
    vcf.all.fb <- rbind(vcf.all.fb, vcf_df)
    
}

# separate tool and file columns
vcf.all.fb <- separate(vcf.all.fb, source_ID, sep = "/", 
               into = c("rep","tool","file"))

# get source ID
vcf.all.fb <- separate(vcf.all.fb, file, sep = "_fixed", 
               into = c("source_ID","rest")) %>% 
           select(-c(rest)) %>% 
           droplevels()

# change class of columns where necessary
vcf.all.fb$gt_DP = as.numeric(vcf.all.fb$gt_DP)
vcf.all.fb$AF = as.numeric(vcf.all.fb$AF)
vcf.all.fb$AF <- round(vcf.all.fb$AF, digits = 4)

vcf.all.fb <- vcf.all.fb %>% select(tool, source_ID, POS, REF, ALT, AF, gt_DP, rep)
# set names to match other data frames
names(vcf.all.fb) <- c("tool","source_ID","ntpos","refnt","varnt","varfreq","totalcount","rep")

vcf.all.fb <- vcf.all.fb %>% filter(varnt %in% ntlist)

[1] "rep1/freebayes/694_fixed.cov19.sorted.merged_freebayes-default.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element GL

Extracting gt element DP

Extracting gt element AD

Extracting gt element RO

Extracting gt element QR

Extracting gt element AO

Extracting gt element QA

Extracting gt element MIN_DP



[1] "rep1/freebayes/OMS-CA-013_fixed.cov19.sorted.merged_freebayes-default.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element GL

Extracting gt element DP

Extracting gt element AD

Extracting gt element RO

Extracting gt element QR

Extracting gt element AO

Extracting gt element QA

Extracting gt element MIN_DP

Warning message:
“Expected 3 pieces. Additional pieces discarded in 1 rows [1].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 1 rows [1].”


[1] "rep1/freebayes/OMS-CA-016_fixed.cov19.sorted.merged_freebayes-default.vcf"


...

Extracting gt element GT

Extracting gt element GQ

Extracting gt element GL

Extracting gt element DP

Extracting gt element AD

Extracting gt element RO

Extracting gt element QR

Extracting gt element AO

Extracting gt element QA

Extracting gt element MIN_DP

Warning message:
“Expected 3 pieces. Additional pieces discarded in 6 rows [1, 8, 21, 22, 41, 47].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 6 rows [1, 8, 21, 22, 41, 47].”


[1] "rep2/freebayes/OMS-CA-629_fixed_freebayes-default.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element GL

Extracting gt element DP

Extracting gt element AD

Extracting gt element RO

Extracting gt element QR

Extracting gt element AO

Extracting gt element QA

Extracting gt element MIN_DP

Warning message:
“Expected 3 pieces. Additional pieces discarded in 4 rows [2, 10, 26, 32].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 4 rows [2, 10, 26, 32].”


[1] "rep2/freebayes/OMS-CA-636_fixed_freebayes-default.vcf"


Extracting gt element GT

Extracting gt element GQ

Extracting gt element GL

Extracting gt element DP

Extracting gt element AD

Extracting gt element RO

Extracting gt element QR

Extracting gt element AO

Extracting gt element QA

Extracting gt element MIN_DP

Warning message:
“Expected 3 pieces. Additional pieces discarded in 5 rows [2, 3, 5, 6, 8].”
Warning message:
“Expected 2 pieces. Additional pieces discarded in 5 rows [2, 3, 5, 6, 8].”


#### All other VCF files - ivar and haplotype caller

In [56]:
# set up dataframe to hold vcf files for timo, haplotype caller and ivar
vcf.all1 <- data.frame()


for (filename in vcf_lists) {
    # read in a file
    vcf <- read.vcfR(filename, verbose = FALSE)
    # convert to a usable data frame
    vcf_df <- vcfR2tidy(vcf,single_frame = TRUE)
    vcf_df <- as.data.frame(vcf_df[[1]])
    
    # make source ID column
    vcf_df$source_ID <- filename
    
    # select desired columns in order
    vcf_df <- vcf_df %>% select(source_ID, POS, REF, ALT, QUAL, FILTER, AF, gt_AD, gt_DP)
    
    # add to larger data frame
    vcf.all1 <- rbind(vcf.all1, vcf_df)
    
}

# separate tool and file columns
vcf.all1 <- separate(vcf.all1, source_ID, sep = "/", 
               into = c("rep","tool","file"))

# get source ID
vcf.all1 <- separate(vcf.all1, file, sep = "_fixed", 
               into = c("source_ID","rest")) %>% 
           select(-c(rest)) %>% 
           droplevels()

# change class of columns where necessary
vcf.all1$gt_DP = as.numeric(vcf.all1$gt_DP)
vcf.all1$AF = as.numeric(vcf.all1$AF)
vcf.all1$AF <- round(vcf.all1$AF, digits = 4)

# set names to match other data frames
names(vcf.all1) <- c("rep","tool","source_ID","ntpos","refnt","varnt","qual","filter","varfreq","allele_depth","totalcount")

# use allele depth column to calculate variant frequency
vcf.all1 <- separate(vcf.all1, allele_depth, sep = ',', into = c("major_dp","minor_dp"))
vcf.all1$minor_dp <- as.numeric(vcf.all1$minor_dp)
# variant freq is equal to number of reads with the alt nucleotide over the total number of reads
vcf.all1 <- vcf.all1 %>%
    mutate(AF2 = minor_dp/totalcount)
vcf.all1$AF2 <- as.numeric(vcf.all1$AF2)
# round decimals to 4 digits
vcf.all1$AF2 <- round(vcf.all1$AF2, digits = 4)
# replace varfreq column with AF2 value
vcf.all1 <- vcf.all1 %>% mutate(varfreq = AF2)
# select columns to match other data sets
vcf.all1 <- vcf.all1 %>% select(tool, source_ID, ntpos, refnt, varnt, varfreq, totalcount, rep)

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting gt element GT

Extracting gt element PL

Extracting gt element AD

Extracting gt element DP

Extracting gt element GQ

Extracting g

In [57]:
head(vcf.all1)
head(vcf.all.fb)
head(vcf.all.t)
head(vcf.mutect)
head(vcf.varscan)
head(vcf.lofreq)

,tool,source_ID,ntpos,refnt,varnt,varfreq,totalcount,rep
,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,filtered_snps,694,241,C,T,0.9990,1963,rep1
2,filtered_snps,694,551,C,T,0.0200,2502,rep1
3,filtered_snps,694,913,C,T,0.0040,1486,rep1
4,filtered_snps,694,1059,C,T,1.0000,1057,rep1
5,filtered_snps,694,2251,A,G,0.0025,6070,rep1
6,filtered_snps,694,2665,T,C,0.9999,6706,rep1


,tool,source_ID,ntpos,refnt,varnt,varfreq,totalcount,rep
,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,freebayes,694,29,A,C,0.0143,280,rep1
2,freebayes,694,241,C,T,0.9995,2157,rep1
3,freebayes,694,1059,C,T,1.0000,1087,rep1
4,freebayes,694,2665,T,C,0.9991,22831,rep1
5,freebayes,694,3037,C,T,0.9996,22390,rep1
6,freebayes,694,3615,T,C,0.0414,266,rep1


,tool,source_ID,ntpos,refnt,varnt,varfreq,totalcount,rep
,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,new_timo,694,1059,C,T,1.000,3257,rep1
2,new_timo,694,7348,T,A,1.000,2,rep1
3,new_timo,694,8083,G,A,1.000,8,rep1
4,new_timo,694,10319,C,T,0.999,37468,rep1
5,new_timo,694,1059,C,T,1.000,3257,rep1
6,new_timo,694,7348,T,A,1.000,2,rep1


,tool,source_ID,ntpos,refnt,varnt,varfreq,totalcount,rep
,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,mutect2,694,241,C,T,0.9995,1926,rep1
2,mutect2,694,1059,C,T,1.0000,1044,rep1
3,mutect2,694,2176,A,G,0.0019,6456,rep1
4,mutect2,694,2461,T,C,0.0017,6416,rep1
5,mutect2,694,2637,T,C,0.0015,6612,rep1
6,mutect2,694,2665,T,C,0.9993,6671,rep1


,tool,source_ID,ntpos,refnt,varnt,varfreq,totalcount,rep
,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,varscan,694,1059,C,T,1.0000,897,rep1
2,varscan,694,2461,T,C,0.0031,7147,rep1
3,varscan,694,2665,T,C,0.9990,7177,rep1
4,varscan,694,3037,C,T,0.9990,7262,rep1
5,varscan,694,3045,C,T,0.0060,7525,rep1
6,varscan,694,5532,G,A,0.0146,6971,rep1


,tool,source_ID,ntpos,refnt,varnt,varfreq,totalcount,rep
,<chr>,<chr>,<int>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,lofreq,694,241,C,T,0.9995,2146,rep1
2,lofreq,694,1059,C,T,1.0000,1085,rep1
3,lofreq,694,2665,T,C,0.9990,22780,rep1
4,lofreq,694,3037,C,T,0.9985,22395,rep1
5,lofreq,694,3045,C,T,0.0038,22372,rep1
6,lofreq,694,3615,T,C,0.0418,263,rep1


In [58]:
# combine all the VCF files from each tools
vcf.all <- do.call("rbind", list(vcf.all1, vcf.all.fb, vcf.all.t, vcf.mutect, vcf.varscan, vcf.lofreq))

In [ ]:
write_excel_csv(x = vcf.all, file = glue("{project_folder}/compare.callers.vcfs.csv"))